In [ ]:
# modules
import pandas as pd
import re

전처리1 - '내용' 영문,일문 버전 제거 및 '제목' 시인정보 제거

In [ ]:
df = pd.read_csv('poems_4151.csv', encoding='cp949')

인코딩(Encoding)
: 문자 -> 숫자 vector 변환

1. ***utf-8***: 전세계 표준. 한글 3바이트 변환
2. ***utf-8-sig***: BOM(Byte Order Mark) 서명 3바이트 + utf-8 버전. BOM 추가로 utf-8 인식 정확도 보장(windows 호환성 강화)
3. ***cp949***: windows 기본 한국 인코딩. 한글 완성형 2바이트 표현

인코딩 확인 방법: 메모장으로 열기 → 다른 이름으로 저장 → 인코딩 확인

In [ ]:
# 제목 정리
def clean_title(title):
    title = title.split('/')[0].strip()
    title = re.sub(r'\(.*?\)', '', title)  # 한자 괄호 제거
    return title.strip()

df['제목'] = df['제목'].apply(clean_title)

In [ ]:
# 내용 정리(한문 버전만 남도록)
def keep_korean_section(text):
    lines = text.strip().split('\n')
    new_lines = []
    for line in lines:
        if re.match(r'^\s*[-‐‑‒–—―ー]{3,}\s*$', line): # 공백 고려 포함
          break
        new_lines.append(line)
    return '\n'.join(new_lines)

In [ ]:
# '내용' 데이터의 마지막 줄 날짜 정보 제거
def remove_trailing_date(text):
    lines = text.strip().split('\n')
    if lines and re.match(r'^\d{4}[,.]\d{1,2}[,.]\d{1,2}$', lines[-1]):
        lines.pop()
    return '\n'.join(lines)

In [ ]:
def pp_ver1_content(text):
    text = keep_korean_section(text) # 영문/일문 제거
    text = remove_trailing_date(text) # 마지막줄 날짜 제거
    return text

In [ ]:
df['내용'] = df['내용'].apply(pp_ver1_content)
df.to_csv("poems_pp_ver1.csv", index=False, encoding='utf-8-sig')

전처리2 - '제목', '내용' 중복 데이터 제거 및 시인 정보 제거

In [ ]:
df_pp_ver1 = pd.read_csv('poems_pp_ver1.csv', encoding='utf-8')

In [ ]:
def handling_content(title, content):
    lines = content.strip().split('\n') # content 줄 단위 쪼개기(lines = list[str])
    cleaned_lines = []
    skip = 0

    # 불필요 데이터(제목, 시인, 한자 정보) 제거
    for i, line in enumerate(lines):
        # line = 리스트(lines) 中 한 줄(lines[i])
        line_clean = line.strip() # line 공백 제거

        # 검사 대상: 첫번째줄 ~ 세번째줄
        if skip < 3:
            # 경우1) 제목 일치
            if line_clean == title:
                skip += 1
                continue
            # 경우2) 제목+한문
            if re.sub(r'[^\w가-힣]', '', line_clean) == re.sub(r'[^\w가-힣]', '', title):
                skip += 1
                continue
            # 경우3) 제목/~
            if line_clean.startswith(title) and '/' in line_clean:
                skip += 1
                continue
            # 경우4) 제목과 관계없이 한글/한자/이름/공백으로만
            if re.match(r'^[가-힣ㄱ-ㅎㅏ-ㅣ·/\s一-龥]+$', line_clean):
                skip += 1
                continue
            # 경우5) 제목(한문)
            if re.match(rf'^{re.escape(title)}\([一-龥]+\)$', line_clean):
                skip += 1
                continue


        # '제목' 중복 데이터 제거한 '내용' 데이터(cleaned_lines) 저장
        cleaned_lines.append(line_clean)

    return '\n'.join(cleaned_lines)

In [ ]:
df_pp_ver1['내용'] = df_pp_ver1.apply(
    lambda x: handling_content(x['제목'], x['내용']),
    axis=1)
    # lambda x, axis=1 => 각 행(Series)마다 함수(중복 데이터 검사) 실행

# 빈 문자열 제거
df_pp_ver1 = df_pp_ver1[df_pp_ver1['내용'].str.strip() != '']

df_pp_ver1.to_csv("poems_pp_ver2.csv", index=False, encoding='utf-8-sig')